### 사용자 정의 데이터셋과 모델과 학습
- iris.csv		=> 사용자 정의 데이터셋
- DNN 모델		=> 사용자 정의 모델


#### [1] 모듈 로딩 & 데이터 준비<hr>

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchmetrics.classification import F1Score
from torchinfo import summary

from torch.utils.data import Dataset, DataLoader

import pandas as pd
from sklearn.preprocessing import LabelEncoder			# 타겟 칼럼 수치화 (인코딩)

In [29]:
# 데이터 
DATA_FILE = r'C:\Users\KDP-43\Desktop\머신러닝_1\data\iris.csv'

In [30]:
irisDF = pd.read_csv(DATA_FILE)
irisDF.head(3)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa


In [31]:
# 타겟 컬럼 수치화 ==> LabelEncoder
encoder = LabelEncoder()
encoder.fit(irisDF['variety'])
irisDF['variety'] = encoder.transform(irisDF['variety'])

print(irisDF.head(3))

   sepal.length  sepal.width  petal.length  petal.width  variety
0           5.1          3.5           1.4          0.2        0
1           4.9          3.0           1.4          0.2        0
2           4.7          3.2           1.3          0.2        0


#### [2] 사용자 정의 데이터셋 클래스 생성<hr>

In [32]:
# --------------------------------------------------
# 클래스 목적: 학습용 데이터셋 텐서화 & 전처리
# 클래스 이름: CustomDataSet
# 부모 클래스: torch.utils.data.DataSet
# 매개	 변수: featureDF, targetDF
# --------------------------------------------------

class CustomDataSet(Dataset):
    
	# 데이터 로딩 & 전처리 & 인스턴스 생성 메서드
    def __init__(self, featureDF, targetDF):
        super().__init__()
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[1]
    
	# 데이터 개수 반환 메서드
    def __len__(self):
        return self.n_rows
    
	# 특정 index 데이터 & 타겟 반환
    # Tensor 형태로
    def __getitem__(self, idx):
        featureTS = torch.FloatTensor( self.featureDF.iloc[idx].values )	# 시리즈 -> array -> tensor
        targetTS = torch.FloatTensor( self.targetDF.iloc[idx].values)
        return featureTS, targetTS
    


In [33]:
# --------------------------------------------------
# 함수 목적: 파일 확장자별 DF 로드 & 변환
# 함수 이름: convertDataFrame
# 매개 변수: file_path
# 함수 결과: DataFrame
# --------------------------------------------------

def convertDataFrame(file_path, is_header=0):
    
	ext = file_path.rsplit('.')[-1]

	if ext == 'csv':
		return pd.read_csv(file_path, header=is_header)
	
	elif ext == 'json':
		return pd.read_json(file_path, header=is_header)
	
	elif ext in ['xlsx', 'xls']:
		return pd.read_excel(file_path, header=is_header)
	
	else:
		return pd.read_table(file_path, header=is_header )
	

    


In [34]:
# --------------------------------------------------
# 클래스 목적: 파일 기반 학습용 데이터셋 텐서화 & 전처리
# 클래스 이름: FileDataSet
# 부모 클래스: torch.utils.data.DataSet
# 매개	 변수: file_path
# --------------------------------------------------

class FileDataSet(Dataset):
    
	# 데이터 로딩 & 전처리 & 인스턴스 생성 메서드
    def __init__(self, file_path):
        super().__init__()

        dataDF = convertDataFrame(file_path)
        self.featuresDF = dataDF[dataDF.columns[:-1]]
        self.targetDF = dataDF[dataDF.columns[-1]]
        
        self.n_features = self.featuresDF.shape[1]
        self.n_rows = self.featuresDF.shape[0]
        
    
	# 데이터 개수 반환 메서드
    def __len__(self):
        return self.n_rows
    
	# 특정 index 데이터 & 타겟 반환
    # Tensor 형태로
    def __getitem__(self, idx):
        featureTS = torch.FloatTensor( self.featureDF.iloc[idx].values )	# 시리즈 -> array -> tensor
        targetTS = torch.FloatTensor( self.targetDF.iloc[idx].values)
        return featureTS, targetTS
    


#### [3] 데이터셋 인스턴스 생성 <hr>

In [35]:
featureDF, targetDF = irisDF[irisDF.columns[:-1]], irisDF[[irisDF.columns[-1]]]
print(f'featureDF => {featureDF.shape}, targetDF => {targetDF.shape}')

featureDF => (150, 4), targetDF => (150, 1)


In [36]:
# IRIS 데이터셋 인스턴스 생성
irisDS = CustomDataSet(featureDF, targetDF)

In [37]:
irisDS.featureDF

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [38]:
irisDS[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0.]))

#### [4] 데이터로더 인스턴스 생성
- iterator 기능 있음

In [39]:
## 필요한 것: Dataset 인스턴스, Batch_size (default=1)
irisDL = DataLoader(irisDS, batch_size=5)

In [40]:
for dataTS, targetTS in irisDL:
    print("dataTS shape", dataTS.shape)
    print("targetTS shape", targetTS.shape)
    break

dataTS shape torch.Size([5, 4])
targetTS shape torch.Size([5, 1])


#### [5] 모델 준비

In [41]:
# ---------------------------------------------------------
# 모델이름: CustomModel
# 부모클래스: nn.Module
# 매개변수: None
# 모델구조
# 	- 입력층: 입력 4개		출력: 10개			AF: ReLU
# 	- 은닉층: 입력 10개		출력: 30개			AF: ReLU		-> leakyReLU 외 적합한 AF 적용!
# 	- 출력층: 입력 30개		출력: 3개			AF: 다중분류 -> softmax
# 					 (0, rest) (1, rest) (2, rest)
# ---------------------------------------------------------

class CustomModel(nn.Module):
    
	# 모델 구조 및 인스턴스 생성 메서드
	def __init__(self):
		super().__init__()

		self.in_layer = nn.Linear(4, 10)
		self.hidden_layer = nn.Linear(10, 30)
		self.out_layer = nn.Linear(30, 3)


	# 순방향 학습 메서드
	def forward(self, x):
		y = F.relu( self.in_layer(x))
		y = F.relu( self.hidden_layer(y))
		
		return self.out_layer(y)
	
	#-------------------------------------------------------
	# 원래는 F.softmax(self.out_layer(y))를 반환해야 하지만!
	# 손실함수 crossentropyloss가가 공식 문서 상,
	# 내부에서 softmax 처리한 확률 반환하기에
	# 출력층 값 그대로 반환
	#-------------------------------------------------------


In [42]:
# 모델 인스턴스 생성
m1 = CustomModel()

# 모델 구조 확인
print(m1)
summary(m1, input_size=(1000, 4))

CustomModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=3, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
CustomModel                              [1000, 3]                 --
├─Linear: 1-1                            [1000, 10]                50
├─Linear: 1-2                            [1000, 30]                330
├─Linear: 1-3                            [1000, 3]                 93
Total params: 473
Trainable params: 473
Non-trainable params: 0
Total mult-adds (M): 0.47
Input size (MB): 0.02
Forward/backward pass size (MB): 0.34
Params size (MB): 0.00
Estimated Total Size (MB): 0.36

#### [6] 학습 진행

In [ ]:
# 테스트 함수

def 

In [54]:
TS_loss, TS_score = {'Train': [], 'VAL':[]}, {'Train': [], 'VAL':[]}

for epoch in range(10):
    
	loss_total, score_total = 0, 0
	# 배치크기만큼 데이터&타겟 추출 & 학습 진행
	for dataTS, targetTS in irisDL:
		
		# 배치크기만큼의 학습 데이터------------------------------------------------
		print(dataTS.shape, targetTS.shape)
		# targetTS = targetTS.reshape(-1).long()		# -> 손실함수 공식문서에 따라 적절한 shape 변환
		
		
		# 배치크기 만큼 학습 진행---------------------------------------------------
		pre_y = m1(dataTS)
		# print(pre_y.shape, targetTS.shape)
		
		# 손실 계산-----------------------------------------------------------------
		# 	=> 분류 ==> crossentropyloss
		loss = nn.CrossEntropyLoss()(pre_y, targetTS)
		loss_total += loss.item()
		# print(loss_total)

		# 모델 점수 계산-------------------------------------------------------------
		score = F1Score()(pre_y, targetTS)
		score_total += score.item()

	TS_loss['Train'].append(loss_total/dataTS.shape[0])
	
	TS_score['Train'].append(score_total/dataTS.shape[0])



torch.Size([5, 4]) torch.Size([5, 1])
torch.Size([5, 3]) torch.Size([5])
0.9509735107421875
torch.Size([5, 4]) torch.Size([5, 1])
torch.Size([5, 3]) torch.Size([5])
1.904440999031067
torch.Size([5, 4]) torch.Size([5, 1])
torch.Size([5, 3]) torch.Size([5])
2.8536389470100403
torch.Size([5, 4]) torch.Size([5, 1])
torch.Size([5, 3]) torch.Size([5])
3.800329625606537
torch.Size([5, 4]) torch.Size([5, 1])
torch.Size([5, 3]) torch.Size([5])
4.751137554645538
torch.Size([5, 4]) torch.Size([5, 1])
torch.Size([5, 3]) torch.Size([5])
5.6948347091674805
torch.Size([5, 4]) torch.Size([5, 1])
torch.Size([5, 3]) torch.Size([5])
6.638330578804016
torch.Size([5, 4]) torch.Size([5, 1])
torch.Size([5, 3]) torch.Size([5])
7.587671518325806
torch.Size([5, 4]) torch.Size([5, 1])
torch.Size([5, 3]) torch.Size([5])
8.538278639316559
torch.Size([5, 4]) torch.Size([5, 1])
torch.Size([5, 3]) torch.Size([5])
9.488984823226929
torch.Size([5, 4]) torch.Size([5, 1])
torch.Size([5, 3]) torch.Size([5])
10.63090503215

In [47]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
print(input.shape, input)
print(target.shape, target)

loss(input, target)
# output.backward()


torch.Size([3, 5]) tensor([[ 0.3981, -1.3182,  0.0930, -0.4525, -0.7986],
        [ 0.5984, -0.2145, -3.7729, -1.0473, -0.9930],
        [ 1.3341, -1.5471, -0.7069,  0.4200,  0.4927]], requires_grad=True)
torch.Size([3]) tensor([2, 1, 0])


tensor(1.1366, grad_fn=<NllLossBackward0>)

In [49]:
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
# target = torch.randn(3, 5).softmax(dim=1)
target = torch.randn(3, 5).argmax(dim=1)

print(input.shape, input)
print(target.shape, target)

loss(input, target)
# output.backward()

torch.Size([3, 5]) tensor([[-1.3592,  0.4736, -0.4823, -0.1302,  0.1927],
        [-1.5477, -1.6713, -0.7099, -0.7827,  2.0851],
        [-0.7363, -1.5797, -1.4759, -1.9363, -0.5215]], requires_grad=True)
torch.Size([3]) tensor([0, 3, 1])


tensor(2.6610, grad_fn=<NllLossBackward0>)